In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
#torch.use_deterministic_algorithms(True)


In [4]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

In [5]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.01
Epochs : 100


In [6]:
cnt = 0
for indent in sorted(list(categories_per_indent_dict.keys())):
    for category in categories_per_indent_dict[indent]:
        cnt+=1

cnt

350

In [7]:
def hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path):

    categories_counter = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            print(f'indent: {indent}')
            category_name = category_id_to_name_dict[category]
            print(f'category_name: {category_name}')

            categories_counter+= 1
            print(f'categories_counter: {categories_counter}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                print(f'category with 0 loss coef: {category_name}|{category}')
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                loss_coef = 1*np.exp(-10)
                parent_model = Model
                parent_optimizer = Optimizer
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)
 
            
            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = Model
            optimizer = Optimizer
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            training_and_evaluation(model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
                


In [8]:
hgru_models = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath)

indent: 0.0
category_name: All items
categories_counter: 1
category with 0 loss coef: All items|8106.0
indent: 1.0
category_name: All items less energy
categories_counter: 2
indent: 1.0
category_name: All items less food
categories_counter: 3
indent: 1.0
category_name: All items less food and energy
categories_counter: 4
indent: 1.0
category_name: All items less food and shelter
categories_counter: 5
indent: 1.0
category_name: All items less food, shelter, and energy
categories_counter: 6
indent: 1.0
category_name: All items less food, shelter, energy, and used cars and trucks
categories_counter: 7
indent: 1.0
category_name: All items less medical care
categories_counter: 8
indent: 1.0
category_name: All items less shelter
categories_counter: 9
indent: 1.0
category_name: Apparel less footwear
categories_counter: 10
indent: 1.0
category_name: Commodities
categories_counter: 11
indent: 1.0
category_name: Durables
categories_counter: 12
indent: 1.0
category_name: Education and communicati

Save prediction dict:

In [9]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model = Model
        basic_optimizer = Optimizer
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [10]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])
    
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Admission to movies, theaters, and concerts
Bacon, breakfast sausage, and related products
Bread other than white
Canned fruits
Canned vegetables
Checking account and other bank services
Chicken
Cookies
Crackers, bread, and cracker products
Dried beans, peas, and lentils
Frankfurters
Fresh cakes and cupcakes
Fresh fish and seafood
Fresh sweetrolls, coffeecakes, doughnuts
Frozen and refrigerated bakery products, pies, tarts, turnovers
Frozen vegetables
Ham
Intracity mass transit
Lunchmeats
Oranges, including tangerines
Other condiments
Other pork including roasts and picnics
Other uncooked poultry including turkey
Parking fees and tolls
Pork chops
Prepared salads
Processed fish and seafood
Salt and other seasonings and spices
Sauces and gravies
Ship fare
Uncooked beef roasts
Uncooked beef steaks
Uncooked ground beef
Uncooked other beef and veal
Video discs and other media
White bread
Admissions
Airline fare
Apparel services other than laundry and dry cleaning
Apples
Bananas
Beef and vea

In [11]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Save model weights in dict for bi-directional use:

In [12]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])

In [13]:
def get_weights_per_category(category_id_list, dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    #basic_model.to(device)

    best_models_weights_dict = {}

    for category_id in category_id_list:
        category_name = category_id_to_name_dict[category_id]
        ckp_path = dir_path+category_name+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        category_model_weights = unify_model_weights(model)
        best_models_weights_dict[category_id] = category_model_weights
        
    return best_models_weights_dict

In [14]:
dir_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/models_weights/'

In [15]:
category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])


In [16]:
len(categories)

350

In [17]:
weights_dict = get_weights_per_category(category_id_list, dir_path)

In [18]:
with open(hgru_model_weights_path, 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
Lr

0.01